In [1]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os, gc
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, GroupKFold
import timm
import random

from torch.optim.lr_scheduler import OneCycleLR
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.functional import log_softmax, softmax
import albumentations as A

import sys
sys.path.append('/content/drive/MyDrive/HMS')
from kaggle_kl_div import score

import warnings

# FutureWarningを無視する
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
class Config:
    def __init__(self, seed=42):
        # 乱数シードの設定
        self.seed = seed
        self.initialize_random_seeds()
        self.initialize_train_param()
        # その他の設定
        self.USE_KAGGLE_SPECTROGRAMS = True
        self.USE_EEG_SPECTROGRAMS = True
        self.DEBUG = False
        self.DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print(f"Using {self.DEVICE}")
        self.ROOT_PATH = '/content/drive/MyDrive/HMS'

    def initialize_random_seeds(self):
        # random
        random.seed(self.seed)
        # numpy
        np.random.seed(self.seed)
        # pytorch
        torch.manual_seed(self.seed)
        torch.cuda.manual_seed_all(self.seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    def initialize_train_param(self):
        self.EPOCHS = 4
        self.N_SPLITS = 5
        self.BATCH_SIZE = 32
        self.AUGMENT = True
config = Config(42)

Using cuda:0


In [5]:
df = pd.read_csv(config.ROOT_PATH + '/train.csv')
TARGETS = df.columns[-6:]

In [6]:
train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spec_id','min']

tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['max'] = tmp

tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values

y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )

Train non-overlapp eeg_id shape: (17089, 12)


In [7]:
%%time
READ_SPEC_FILES = False

# READ ALL SPECTROGRAMS
PATH = config.ROOT_PATH + '/train_spectrograms/'
files = os.listdir(PATH)
print(f'There are {len(files)} spectrogram parquets')

if READ_SPEC_FILES:
    spectrograms = {}
    for i,f in enumerate(files):
        if i%100==0: print(i,', ',end='')
        tmp = pd.read_parquet(f'{PATH}{f}')
        name = int(f.split('.')[0])
        spectrograms[name] = tmp.iloc[:,1:].values
else:
    spectrograms = np.load(config.ROOT_PATH  + '/specs.npy',allow_pickle=True).item()

There are 0 spectrogram parquets
CPU times: user 693 ms, sys: 6.7 s, total: 7.4 s
Wall time: 1min 52s


In [8]:
%%time
READ_EEG_SPEC_FILES = False

if READ_EEG_SPEC_FILES:
    all_eegs = {}
    for i,e in enumerate(train.eeg_id.values):
        if i%100==0: print(i,', ',end='')
        x = np.load(config.ROOT_PATH + f'/EEG_Spectrograms/{e}.npy')
        all_eegs[e] = x
else:
    all_eegs = np.load(config.ROOT_PATH + '/eeg_specs.npy',allow_pickle=True).item()

CPU times: user 1.16 s, sys: 8.95 s, total: 10.1 s
Wall time: 2min 24s


In [9]:
TARS = {'Seizure': 0, 'LPD': 1, 'GPD': 2, 'LRDA': 3, 'GRDA': 4, 'Other': 5}
TARS2 = {x:y for y,x in TARS.items()}


class EfficentNetDataset(Dataset):
    def __init__(self, data, augment=config.AUGMENT, mode='train',
                 specs = spectrograms, eeg_specs = all_eegs):

        self.data = data
        self.augment = augment
        self.mode = mode
        self.specs = specs
        self.eeg_specs = eeg_specs
#         self.on_epoch_end()
        self.indexes = np.arange( len(self.data))
    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        indexes = self.indexes[idx]
        X, y = self.__data_generation(indexes)
        if self.augment:
            X = self._augment_batch(X)
        if self.mode != 'test':
            return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)
        else:
            return torch.tensor(X, dtype=torch.float32)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples'

        X = np.zeros((128,256,8),dtype='float32')
        y = np.zeros((6),dtype='float32')
        img = np.ones((128,256),dtype='float32')

        row = self.data.iloc[indexes]
        if self.mode=='test':
            r = 0
        else:
            r = int( (row['min'] + row['max'])//4 )

        for k in range(4):
            # EXTRACT 300 ROWS OF SPECTROGRAM(4種類抜いてくる)
            img = self.specs[row.spec_id][r:r+300,k*100:(k+1)*100].T

            # LOG TRANSFORM SPECTROGRAM
            img = np.clip(img,np.exp(-4),np.exp(8))
            img = np.log(img)

            # STANDARDIZE PER IMAGE
            ep = 1e-6
            m = np.nanmean(img.flatten())
            s = np.nanstd(img.flatten())
            img = (img-m)/(s+ep)
            img = np.nan_to_num(img, nan=0.0)

            # CROP TO 256 TIME STEPS
            X[14:-14,:,k] = img[:,22:-22] / 2.0

        # EEG SPECTROGRAMS
        img = self.eeg_specs[row.eeg_id]
        X[:,:,4:] = img

        if self.mode!='test':
            y = row.loc[TARGETS]

        return X,y

    def _augment_batch(self, img):
        transforms = A.Compose([
#             A.HorizontalFlip(p=0.2),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0, rotate_limit=0, p=0.3, border_mode=0), # 時間軸方向のシフト
            A.GaussNoise(var_limit=(10, 50), p=0.3) # ガウス雑音
        ])
        return transforms(image=img)['image']

In [10]:
class CustomInputTransform(nn.Module):
    def __init__(self, use_kaggle=True, use_eeg=True):
        super(CustomInputTransform, self).__init__()
        self.use_kaggle = use_kaggle
        self.use_eeg = use_eeg

    def forward(self, x):
        # Kaggleスペクトログラム
        if self.use_kaggle:
            x1 = torch.cat([x[:, :, :, i:i+1] for i in range(4)], dim=1)

        # EEGスペクトログラム
        if self.use_eeg:
            x2 = torch.cat([x[:, :, :, i+4:i+5] for i in range(4)], dim=1)

        # 結合
        if self.use_kaggle and self.use_eeg:
            x = torch.cat([x1, x2], dim=2)
        elif self.use_eeg:
            x = x2
        else:
            x = x1

        # 3チャンネルに複製
        x = x.repeat(1, 1, 1, 3)
        x = x.permute(0, 3, 1, 2)

        return x

class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes=6):
        super(CustomEfficientNet, self).__init__()
        self.input_transform = CustomInputTransform(use_kaggle=True, use_eeg=True)
        self.base_model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=0, in_chans=3)
        # 他の層の定義...

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(in_features=1280, out_features=num_classes) #1280 #1408 #1792
        self.base_model.classifier = self.fc

    def forward(self, x):
        x = self.input_transform(x)
        x = self.base_model(x)
        return x

In [11]:
if config.DEBUG:
    train = train.iloc[:200]

In [12]:
def calc_cv_score(oof,true):
    oof = pd.DataFrame(np.concatenate(oof).copy())
    oof['id'] = np.arange(len(oof))

    true = pd.DataFrame(np.concatenate(true).copy())
    true['id'] = np.arange(len(true))

    cv = score(solution=true, submission=oof, row_id_column_name='id')
    return cv

In [13]:
# モデル訓練関数
def train_model(model, train_loader, test_loader):
    model.train()
    train_loss = []
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(config.DEVICE), labels.to(config.DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(log_softmax(outputs, dim = 1), labels)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
        if config.DEBUG:
            print(f'train_loss: {loss.item()}')
    scheduler.step()
    # 検証ループ
    model.eval()
    oof = []
    true = []
    valid_loss = []
    with torch.no_grad():
        for inputs, labels in valid_loader:
            true.append(labels)
            inputs, labels = inputs.to(config.DEVICE), labels.to(config.DEVICE)
            outputs = model(inputs)
            loss = criterion(log_softmax(outputs, dim = 1), labels)
            valid_loss.append(loss.item())
            oof.append(softmax(outputs,dim=1).to('cpu').numpy())

    # モデルの重みを保存
    cv=calc_cv_score(oof,true)
    return model,np.mean(train_loss),np.mean(valid_loss),cv

In [ ]:
gkf = GroupKFold(n_splits=config.N_SPLITS)
for i, (train_index, valid_index) in enumerate(gkf.split(train, train.target, train.patient_id)):
    print(f'### Fold {i+1}')
    # データローダーの作成
    train_dataset = EfficentNetDataset(train.iloc[train_index])
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=2,pin_memory=True)

    valid_dataset = EfficentNetDataset(train.iloc[valid_index], augment=False)
    valid_loader = DataLoader(valid_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=2,pin_memory=True)

    # モデルの構築
    model = CustomEfficientNet().to(config.DEVICE)
    optimizer = optim.AdamW(model.parameters(),lr=0.001)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=4, eta_min=1e-4)
    criterion = nn.KLDivLoss(reduction='batchmean')  # 適切な損失関数を選択

    # トレーニングループ
    for epoch in range(config.EPOCHS):
        model, tr_loss, val_loss, cv = train_model(model, train_loader, valid_loader)
        # エポックごとのログを出力
        print(f'Epoch {epoch+1}, Train Loss: {tr_loss}, Valid Loss: {val_loss}')
        print('CV Score KL-Div for EfficientNetB2 =',cv)
        torch.save(model.state_dict(), config.ROOT_PATH + f'/model/fold{i}_Eff_net_snapshot_epoch_{epoch}.pickle')
    del model
    gc.collect()
    torch.cuda.empty_cache()

### Fold 1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Epoch 1, Train Loss: 0.9266530002388998, Valid Loss: 0.7708299502591106
CV Score KL-Div for EfficientNetB2 = 0.7703923424441953
Epoch 2, Train Loss: 0.8031856293015391, Valid Loss: 0.8184790472003901
CV Score KL-Div for EfficientNetB2 = 0.8179458187498682
Epoch 3, Train Loss: 0.712386811935456, Valid Loss: 0.7136965583418017
CV Score KL-Div for EfficientNetB2 = 0.7135171363328738
Epoch 4, Train Loss: 0.6433468430398781, Valid Loss: 0.6570088155915804
CV Score KL-Div for EfficientNetB2 = 0.6567210762038204
### Fold 2
Epoch 1, Train Loss: 0.9288738219537468, Valid Loss: 0.8095244458902662
CV Score KL-Div for EfficientNetB2 = 0.8096444522454751
Epoch 2, Train Loss: 0.7939370038492657, Valid Loss: 0.7315949808771365
CV Score KL-Div for EfficientNetB2 = 0.7315769756622592
Epoch 3, Train Loss: 0.7143140114217161, Valid Loss: 0.65278616742553
CV Score KL-Div for EfficientNetB2 = 0.6529567975099776
Epoch 4, Train Loss: 0.6534061277998942, Valid Loss: 0.5900456092067968
CV Score KL-Div for Effi

In [ ]:
from google.colab import runtime
runtime.unassign()